<a href="https://colab.research.google.com/github/Sugina99/Deep_Learning_Studies/blob/main/Multi_Class_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers,Model

import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# ! unzip "/content/drive/MyDrive/LibrariesForAI/Deep_learning/Transfer_Learning/Intel_Image_Multiclass/test.zip" -d "/content/drive/MyDrive/LibrariesForAI/Deep_learning/Transfer_Learning/Intel_Image_Multiclass/test"

In [3]:
# ! unzip "/content/drive/MyDrive/LibrariesForAI/Deep_learning/Transfer_Learning/Intel_Image_Multiclass/train.zip" -d "/content/drive/MyDrive/LibrariesForAI/Deep_learning/Transfer_Learning/Intel_Image_Multiclass/train"
# ! unzip "/content/drive/MyDrive/LibrariesForAI/Deep_learning/Transfer_Learning/Intel_Image_Multiclass/validation.zip" -d "/content/drive/MyDrive/LibrariesForAI/Deep_learning/Transfer_Learning/Intel_Image_Multiclass/validation"

In [4]:
train_path="/content/drive/MyDrive/LibrariesForAI/Deep_learning/Transfer_Learning/Intel_Image_Multiclass/train/seg_train"
validation_path="/content/drive/MyDrive/LibrariesForAI/Deep_learning/Transfer_Learning/Intel_Image_Multiclass/validation/seg_test"

In [5]:
! ls "/content/drive/MyDrive/LibrariesForAI/Deep_learning/Transfer_Learning/Intel_Image_Multiclass/train/seg_train"

buildings  forest  glacier  mountain  sea  street


# Transfer Learning - InceptionResNetV2

In [6]:
# basic parameters
batch_size=32 # power of 2---->32,64,128,256,512
epochs=10
Image_height=150
Image_width=150


### Image Augmentation


In [7]:
# train
train_data_gen=ImageDataGenerator(rescale=1./255,
                                 horizontal_flip=True,
                                 rotation_range=45,
                                 zoom_range=0.2,
                                 height_shift_range=0.2,
                                 width_shift_range=0.2)
# validation                                 
validation_data_gen=ImageDataGenerator(rescale=1./255)

### Traing and Validation set

In [8]:
train_generator=train_data_gen.flow_from_directory(batch_size=batch_size,
                                              target_size=(Image_height,Image_width),
                                              directory=train_path,
                                              shuffle=True,
                                              class_mode="categorical")

Found 14034 images belonging to 6 classes.


In [9]:
validation_generator=validation_data_gen.flow_from_directory(batch_size=batch_size,
                                              target_size=(Image_height,Image_width),
                                              directory=validation_path,
                                              shuffle=True,
                                              class_mode="categorical")

Found 3000 images belonging to 6 classes.


### Loading base model


In [10]:
base_model=tf.keras.applications.InceptionResNetV2(input_shape=(150,150,3),
                             include_top=False,
                             weights="imagenet")

219070464/219055592 [==============================] - 2s 0us/step


In [11]:
# Freezing the CNN layer
# for layer in base_model.layers:
#   layer.trainable=False
# or we can use
base_model.trainable=False

In [12]:
model=tf.keras.Sequential([
    base_model,
    tf.keras.layers.BatchNormalization(renorm=True),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(6,activation='softmax')
])

### Compile and Fit

In [14]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
model.fit(train_generator,
          steps_per_epoch=(14034//batch_size),
          epochs=5,
          validation_data=validation_generator,
          validation_steps=(3000//batch_size))

Epoch 1/5
363/438 [=======================>......] - ETA: 7:21 - loss: 0.6640 - accuracy: 0.7586

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-745a6ff084c5>", line 5, in <module>
    validation_steps=(3000//batch_size))
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/engine/training.py", line 1384, in fit
    tmp_logs = self.train_function(iterator)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 915, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 947, in _call
    return self._state

KeyboardInterrupt: ignored

### Saving Model

In [1]:
model.Save("/content/drive/MyDrive/LibrariesForAI/Deep_learning/Transfer_Learning/Intel_Image_Multiclass/Multiclasses.h5")

NameError: ignored

### Loading Model

In [ ]:
from keras.models import model
model=load_model("/content/drive/MyDrive/LibrariesForAI/Deep_learning/Transfer_Learning/Intel_Image_Multiclass/Multiclasses.h5")

### Testing the model on the test data set

In [ ]:
test_dataset="/content/drive/MyDrive/LibrariesForAI/Deep_learning/Transfer_Learning/Intel_Image_Multiclass/test/seg_pred"

In [ ]:
test_data_gen=ImageDataGenerator(rescale=1./255)
test_generator=test_data_gen.flow_from_directory(batch_size=batch_size,
                                              target_size=(Image_height,Image_width),
                                              directory=train_path,
                                              shuffle=False,
                                              class_mode="categorical")